In [1]:
%matplotlib widget

In [2]:
import nmrProblemGUI

In [3]:
ipwidgetsGUI = nmrProblemGUI.ipywidgetsDisplay()

In [4]:
ipwidgetsGUI

ipywidgetsDisplay(children=(VBox(children=(Accordion(children=(HBox(children=(FileUpload(value={}, description…

ch9_025
created temporary directory C:\Users\ERIC\AppData\Local\Temp\tmphc1rvvc9
C:\Users\ERIC\AppData\Local\Temp\tmphc1rvvc9\ch9_025


C:\Users\ERIC\anaconda3\lib\site-packages\ipysheet\pandas_loader.py:88: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(arr)
